In [10]:
import cv2
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def ScoreCam(model, img_array, layer_name, max_N=-1):

    cls = np.argmax(model.predict(img_array))
    act_map_array = Model(inputs=model.input, outputs=model.get_layer(layer_name).output).predict(img_array)
    
    # extract effective maps
    if max_N != -1:
        act_map_std_list = [np.std(act_map_array[0,:,:,k]) for k in range(act_map_array.shape[3])]
        unsorted_max_indices = np.argpartition(-np.array(act_map_std_list), max_N)[:max_N]
        max_N_indices = unsorted_max_indices[np.argsort(-np.array(act_map_std_list)[unsorted_max_indices])]
        act_map_array = act_map_array[:,:,:,max_N_indices]

    input_shape = model.layers[0].output_shape[0][1:]  # get input shape
    # 1. upsample to original input size
    #INTER_LINEAR : a bilinear interpolation (used by default)
    act_map_resized_list = [cv2.resize(act_map_array[0,:,:,k], input_shape[:2], interpolation=cv2.INTER_LINEAR) for k in range(act_map_array.shape[3])]
    # 2. normalize the raw activation value in each activation map into [0, 1]
    act_map_normalized_list = []
    for act_map_resized in act_map_resized_list:
        if np.max(act_map_resized) - np.min(act_map_resized) != 0:
            act_map_normalized = act_map_resized / (np.max(act_map_resized) - np.min(act_map_resized))
        else:
            act_map_normalized = act_map_resized
        act_map_normalized_list.append(act_map_normalized)
    # 3. project highlighted area in the activation map to original input space by multiplying the normalized activation map
    masked_input_list = []
    for act_map_normalized in act_map_normalized_list:
        masked_input = np.copy(img_array)
        for k in range(3):
            masked_input[0,:,:,k] *= act_map_normalized
        masked_input_list.append(masked_input)
    masked_input_array = np.concatenate(masked_input_list, axis=0)
    # 4. feed masked inputs into CNN model and softmax
    pred_from_masked_input_array = softmax(model.predict(masked_input_array))
    # 5. define weight as the score of target class
    weights = pred_from_masked_input_array[:,cls]
    # 6. get final class discriminative localization map as linear weighted combination of all activation maps
    cam = np.dot(act_map_array[0,:,:,:], weights)
    cam = np.maximum(0, cam)  # Passing through ReLU
    cam /= np.max(cam)  # scale 0 to 1.0
    
    return cam

def read_and_preprocess_img(path, size=(224,224)):
    img = load_img(path, target_size=size)
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x
def superimpose(original_img_path, cam, emphasize=False):
    
    img_bgr = cv2.imread(original_img_path)

    heatmap = cv2.resize(cam, (img_bgr.shape[1], img_bgr.shape[0]))
    if emphasize:
        heatmap = sigmoid(heatmap, 50, 0.5, 1)
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    hif = .8
    superimposed_img = heatmap * hif + img_bgr
    superimposed_img = np.minimum(superimposed_img, 255.0).astype(np.uint8)  # scale 0 to 255  
    superimposed_img_rgb = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    #fin = cv2.addWeighted(heatmap_img, 0.7, img, 0.3, 0)
    
    return superimposed_img_rgb
def softmax(x):
    f = np.exp(x)/np.sum(np.exp(x), axis = 1, keepdims = True)
    return f

In [11]:
def resize(original_img_path, cam, emphasize=False):
    
    img_bgr = cv2.imread(original_img_path)

    heatmap = cv2.resize(cam, (img_bgr.shape[1], img_bgr.shape[0]))
    
    return heatmap

In [12]:
import os
img_basenames = []
for file in os.listdir("H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/n01484850"):
    if file.endswith(".JPEG"):
        #print(file)
        img_basenames.append(file)

In [13]:
len(img_basenames)


1300

In [ ]:
#test for one class, make sure you create the output folder for the produced activation maps (heatmaps)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
import os
os.chdir("R:/Bureau")
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

#from gradcamutils import GradCam, GradCamPlusPlus, ScoreCam, build_guided_model, GuidedBackPropagation, superimpose, read_and_preprocess_img

model = VGG16(include_top=True, weights='imagenet')
layer_name = 'block5_conv3'
#loop for a directory containing at least 1000 images from each class of the imagenet dataset

# choices to add dependending on which dataset
for basename in img_basenames:
    img_path = f'H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/n01484850/{basename}'
    orig_img = np.array(load_img(img_path),dtype=np.uint8)
    img_array = read_and_preprocess_img(img_path, size=(224,224))

    predictions = model.predict(img_array)
    top = decode_predictions(predictions, top=5)[0]

    

    score_cam=ScoreCam(model,img_array,layer_name)
    score_cam_superimposed = superimpose(img_path, score_cam)

    #superimposed cam is scaled from 0 to 255
    plt.imshow(score_cam_superimposed)
    plt.axis('off')
    plt.savefig(f"n01484850_ScoreCAM/result_{top[0][1]}_{basename}")
    

In [14]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
import os
os.chdir("R:/Bureau")
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions

#from gradcamutils import GradCam, GradCamPlusPlus, ScoreCam, build_guided_model, GuidedBackPropagation, superimpose, read_and_preprocess_img

model = VGG16(include_top=True, weights='imagenet')
layer_name = 'block5_conv3'
#loop for a directory containing at least 1000 images from each class of the imagenet dataset

# choices to add dependending on which dataset
for basename in img_basenames:
    img_path = f'H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/n01484850/{basename}'
    orig_img = np.array(load_img(img_path),dtype=np.uint8)
    img_array = read_and_preprocess_img(img_path, size=(224,224))

    predictions = model.predict(img_array)
    top = decode_predictions(predictions, top=5)[0]

    

    print(top[0][1])
    
    

great_white_shark
great_white_shark
great_white_shark
sturgeon
hammerhead
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
scuba_diver
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
grey_whale
great_white_shark
great_white_shark
great_white_shark
hammerhead
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
tiger_shark
great_white_shark
great_white_shark
hammerhead
great_white_shark
great_white_shark
great_white_shark
tiger_shark
great_white_shark
great_white_shark
hammerhead
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
great_white_shark
tiger_shark
great_white_shark
great_white_shark
hammer

KeyboardInterrupt: 

# loop for imagenet

In [17]:
import pandas as pd 
import re

df =  pd.read_csv(
    'IMAGENET_Classes.txt', 
    sep = ',', 
    engine = 'python', 
    header = None, 
    index_col = False, 
    names= [
        "ID","class_name"
    ]
)


FileNotFoundError: [Errno 2] No such file or directory: 'IMAGENET_Classes.txt'

In [4]:
df ['class_name'] = df ['class_name'].str.replace (',',' ')

In [ ]:
for i in range(1000):
    for basename in img_basenames:
        img_path = f'H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/'+df['class_name']+'/'{basename}
        orig_img = np.array(load_img(img_path),dtype=np.uint8)
        img_array = read_and_preprocess_img(img_path, size=(224,224))

        predictions = model.predict(img_array)
        top = decode_predictions(predictions, top=5)[0]

    

        

# Predictions

In [5]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16

import matplotlib.pyplot as plt
import os
os.chdir("R:/Bureau")
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
import pandas as pd
data_count = pd.DataFrame(columns=['ID', 'name', 'TN', 'TP'])
model = VGG16(include_top=True, weights='imagenet')
layer_name = 'block5_conv3'
TP=[]
TN=[]
for i in range(1000):
    img_basenames = []
    directory = df['ID'][i]
    for file in os.listdir(f"H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/"+directory):
        if file.endswith(".JPEG"):
        #print(file)
            img_basenames.append(file)
    for basename in img_basenames:
        img_path = f'H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train/'+directory+'/'+basename
        orig_img = np.array(load_img(img_path),dtype=np.uint8)
        img_array = read_and_preprocess_img(img_path, size=(224,224))

        predictions = model.predict(img_array)
        top = decode_predictions(predictions, top=5)[0]
        #print(top[0][1])
        if (top[0][1]==df['class_name'][i]):
            TP[i] = TP[i]+1
        else:
            TN[i] = TN [i] + 1
        data_count = data_count.append(pd.DataFrame(data={'ID': df['ID'][i], 'name': df['class_name'][i], 'TN': TN, 'TP': TP}, index = [0]), ignore_index = True)
        #score_cam=ScoreCam(model,img_array,layer_name)
        #score_cam_superimposed = superimpose(img_path, score_cam)

        #superimposed cam is scaled from 0 to 255
        #plt.imshow(score_cam_superimposed)
        #plt.axis('off')
        #THE FOLDER SHOULD BE CREATED BEFOREHAND !!!!
        #plt.savefig(basename+"_ScoreCAM/"+top[0][1])

KeyboardInterrupt: 

In [21]:
import tensorflow as tf

In [22]:
print(tf.__version__)

2.4.1


In [6]:
data_count

,ID,name,TN,TP
0,n01440764,tench,1,0
1,n01440764,tench,2,0
2,n01440764,tench,3,0
3,n01440764,tench,4,0
4,n01440764,tench,5,0
...,...,...,...,...
17469,n01534433,junco,14423,3047
17470,n01534433,junco,14424,3047
17471,n01534433,junco,14425,3047
17472,n01534433,junco,14426,3047


In [7]:
import shutil
import os
import numpy as np
import argparse

def get_files_from_folder(path):

    files = os.listdir(path)
    return np.asarray(files)

def main(path_to_data, path_to_test_data, train_ratio):
    # get dirs
    _, dirs, _ = next(os.walk(path_to_data))

    # calculates how many train data per class
    data_counter_per_class = np.zeros((len(dirs)))
    for i in range(len(dirs)):
        path = os.path.join(path_to_data, dirs[i])
        files = get_files_from_folder(path)
        data_counter_per_class[i] = len(files)
    test_counter = np.round(data_counter_per_class * (1 - train_ratio))

    # transfers files
    for i in range(len(dirs)):
        path_to_original = os.path.join(path_to_data, dirs[i])
        path_to_save = os.path.join(path_to_test_data, dirs[i])

        #creates dir
        if not os.path.exists(path_to_save):
            os.makedirs(path_to_save)
        files = get_files_from_folder(path_to_original)
        # moves data
        for j in range(int(test_counter[i])):
            dst = os.path.join(path_to_save, files[j])
            src = os.path.join(path_to_original, files[j])
            shutil.move(src, dst)



In [9]:

if __name__ == "__main__":
    data_path = "H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train"
    test_data_path_to_save = "R:/Bureau/test_data"
    train_ratio = 0.2
    main(data_path, test_data_path_to_save, float(train_ratio))

KeyboardInterrupt: 

In [ ]:
'''
#If you are not too keen on coding, there is a python package called split-folders that you could use. 
#It is extremely easy to use and can be found here Here is how it can be used.

!pip install split_folders
import split-folders
input_folder = "input_path"
output = "output_path" #where you want the split datasets saved. one will be created if none is set

split_folders.ratio('input_folder', output="output", seed=42, ratio=(.8, .1, .1)) # ratio of split are in order of train/val/test. You can change to whatever you want. For train/val sets only, you could do .75, .25 for example.
'''

In [ ]:
''' 

import os
import numpy as np
import shutil
import random
root_dir = "H:/imagenet_object_localization_patched2019/ILSVRC/Data/CLS-LOC/train"
classes_dir = ['0', '1']

test_ratio = 0.20

for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls)
    os.makedirs(root_dir +'test/' + cls)

src = root_dir + cls

allFileNames = os.listdir(src)
np.random.shuffle(allFileNames)
train_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                          [int(len(allFileNames)* (1 - test_ratio))])


train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

print("*****************************")
print('Total images: ', len(allFileNames))
print('Training: ', len(train_FileNames))
print('Testing: ', len(test_FileNames))
print("*****************************")


lab = ['0', '1']

for name in train_FileNames:
    for i in lab:
        shutil.copy(name, root_dir +'train/' + i)

for name in test_FileNames:
    for i in lab:
        shutil.copy(name, root_dir +'test/' + i)
print("Copying Done!")

'''